In [ ]:
from FSH_Functions import *
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import folium
from osgeo import gdal
import rasterio as rio
from folium import plugins
from rasterio.warp import calculate_default_transform, reproject, Resampling
import earthpy as et
import branca
import time

os.getcwd()
os.chdir("../../Retrieved_Data/SAR_CH4/test_example_ROIPAC/")

In [ ]:
in_path = [os.path.join("f890_o118", "890_118_20070808_HV_20070923_HV_fsh.tif"), os.path.join("f890_o119", "890_119_20070710_HV_20071010_HV_fsh.tif"), os.path.join("f890_o120", "890_120_20070727_HV_20070911_HV_fsh.tif")]

In [ ]:
in_path

In [ ]:
print("Reading file %s at " % in_path[1] + time.strftime("%H:%M:%S"))

In [ ]:
viridis = plt.cm.get_cmap('viridis', 256)

driver=gdal.GetDriverByName('GTiff')
driver.Register() 
ds = gdal.Open(in_path[1]) 
if ds is None:
    print('Could not open')

#Get coordinates, cols and rows
geotransform = ds.GetGeoTransform()
cols = ds.RasterXSize 
rows = ds.RasterYSize 

#Get extent
xmin=geotransform[0]
ymax=geotransform[3]
xmax=xmin+cols*geotransform[1]
ymin=ymax+rows*geotransform[5]

#Get Central point
centerx=(xmin+xmax)/2
centery=(ymin+ymax)/2

map= folium.Map(location=[centery, centerx], zoom_start=7)

#Open raster file
for i in in_path:
    print("Reading file %s at " % i + time.strftime("%H:%M:%S"))
    driver=gdal.GetDriverByName('GTiff')
    driver.Register() 
    ds = gdal.Open(i) 
    if ds is None:
        print('Could not open')

    #Get coordinates, cols and rows
    geotransform = ds.GetGeoTransform()
    cols = ds.RasterXSize 
    rows = ds.RasterYSize 

    #Get extent
    xmin=geotransform[0]
    ymax=geotransform[3]
    xmax=xmin+cols*geotransform[1]
    ymin=ymax+rows*geotransform[5]

    #Get Central point
    centerx=(xmin+xmax)/2
    centery=(ymin+ymax)/2

    #Raster convert to array in numpy
    bands = ds.RasterCount
    bands=ds.GetRasterBand(1)
    stats = bands.GetStatistics(0,1)
    bands.SetNoDataValue(255)
    print ("[ STATS ] =  Minimum=%.3f, Maximum=%.3f, Mean=%.3f, StdDev=%.3f" % ( \
                stats[0], stats[1], stats[2], stats[3] ))    
    dataset= bands.ReadAsArray(0,0,cols,rows).astype(np.float32)
    dataimage=dataset
    dataimage[dataimage[:,:]==-340282346638528859811704183484516925440.000]=0
    
    dataimage[dataimage == 255] = np.NaN
    print(dataimage[~np.isnan(dataimage)].mean())
    
    dataimage = dataimage/40
    
    folium.raster_layers.ImageOverlay(
        image=dataimage,
        colormap= plt.cm.viridis,
        bounds=[[ymin, xmin], [ymax, xmax]],
        opacity=0.8,
        name = i,
        interactive=True
    ).add_to(map)
    
    print("File %s completed at " % i + time.strftime("%H:%M:%S"))

folium.LayerControl().add_to(map)

colormap = branca.colormap.linear.viridis.scale(0, 40)
colormap = colormap.to_step(index=np.linspace(0,40,17))
colormap.caption = 'Forest Stand Height'
colormap.add_to(map)


In [ ]:
map

In [ ]:
print("Reading file %s at " % i + time.strftime("%H:%M:%S"))
driver=gdal.GetDriverByName('GTiff')
driver.Register() 
ds = gdal.Open(i) 
if ds is None:
    print('Could not open')
    

#Get coordinates, cols and rows
geotransform = ds.GetGeoTransform()
cols = ds.RasterXSize 
rows = ds.RasterYSize 

#Get extent
xmin=geotransform[0]
ymax=geotransform[3]
xmax=xmin+cols*geotransform[1]
ymin=ymax+rows*geotransform[5]

#Get Central point
centerx=(xmin+xmax)/2
centery=(ymin+ymax)/2

#Raster convert to array in numpy
bands = ds.RasterCount
bands=ds.GetRasterBand(1)
stats = bands.GetStatistics(0,1)
bands.SetNoDataValue(255)
print ("[ STATS ] =  Minimum=%.3f, Maximum=%.3f, Mean=%.3f, StdDev=%.3f" % ( \
                                                                            stats[0], stats[1], stats[2], stats[3] ))    
dataset= bands.ReadAsArray(0,0,cols,rows).astype(np.float32)
dataimage=dataset
    dataimage[dataimage[:,:]==-340282346638528859811704183484516925440.000]=0
    
    dataimage[dataimage == 255] = np.NaN
    print(dataimage[~np.isnan(dataimage)].mean())
    
    dataimage = dataimage/40